# Air Quality Data per US States (Cleaning)

This code cleans and processes air quality data specifically for the United States. The original data consists of multiple CSV files with daily pollutant measurements for various countries. Given the diversity of pollutants and inconsistencies in data coverage, only PM2.5 (Particulate Matter ≤ 2.5 micrometers) is analyzed. Annual exposure for each state is calculated as the mean of daily median PM2.5 values. The final cleaned data is saved as a CSV file with columns for US state, year (YYYY format), and yearly mean PM2.5.


In [1]:
# Dependencies
import pandas as pd
from pathlib import Path
from pprint import pprint

## Load Only Data for PM25 and for the US from the CSV Files 

In [3]:
# Focus on PM2.5 as the air quality metric
aq_metric = 'pm25'

# List of file extensions for datasets
exts = [
    '2015H1', '2016H1', '2017H1', '2018H1',
    '2019Q1', '2019Q2', '2019Q3', '2019Q4',
    '2020Q1', '2020Q2', '2020Q3', '2020Q4',
    '2021Q1', '2021Q2', '2021Q3', '2021Q4'
]

# Load initial file and create base DataFrame
ext = exts[0]
csvfile = Path(f"Raw_Data/Air_Quality_Raw/waqi-covid-{ext}.csv")
data = pd.read_csv(csvfile)

# Filter for PM2.5 data, selecting necessary columns and calculating mean PM2.5 by country and city
reduced_df = (
    data[data['Specie'] == aq_metric]
    .loc[:, ['Date', 'Country', 'City', 'median']]
    .groupby(['Country', 'City']).mean('median')
    .rename(columns={'median': 'PM25'})
)
# Add year column from file name and rearrange columns
reduced_df['Year'] = ext[:4]
reduced_df = reduced_df[['Year', 'PM25']]

reduced_df

Year        PM25
Country City                             
AE      Dubai            2015  118.714286
AT      Linz             2015   53.005376
        Vienna           2015   52.629630
AU      Brisbane         2015   24.947090
        Darwin           2015   26.154286
...                       ...         ...
US      The Bronx        2015   34.031746
        Tucson           2015   25.719577
        Washington D.C.  2015   37.661376
VN      Huế              2015   78.166667
        Hạ Long          2015  145.000000

[380 rows x 2 columns]

In [5]:
# Loop through each file extension to process and append data
for ext in exts[1:]:
    print(f"Processing file: {ext}")  # Display the current file being processed

    # Load the CSV file
    csvfile = Path(f"Raw_Data/Air_Quality_Raw/waqi-covid-{ext}.csv")
    data = pd.read_csv(csvfile)
    
    # Filter for PM2.5, select relevant columns, and calculate mean by country and city
    new_df = (
        data[data['Specie'] == aq_metric]
        .loc[:, ['Date', 'Country', 'City', 'median']]
        .groupby(['Country', 'City']).mean('median')
        .rename(columns={'median': 'PM25'})
    )
    
    # Add the year column based on the file name
    new_df['Year'] = ext[:4]
    
    # Select and reorder columns
    new_df = new_df[['Year', 'PM25']]
    
    # Append the processed data to the main DataFrame
    reduced_df = pd.concat([reduced_df, new_df])

# Final output message and display of the resulting DataFrame's head
print('Data processing complete. Displaying DataFrame:')
reduced_df

Processing file: 2016H1
Processing file: 2017H1
Processing file: 2018H1
Processing file: 2019Q1
Processing file: 2019Q2
Processing file: 2019Q3
Processing file: 2019Q4
Processing file: 2020Q1
Processing file: 2020Q2
Processing file: 2020Q3
Processing file: 2020Q4
Processing file: 2021Q1
Processing file: 2021Q2
Processing file: 2021Q3
Processing file: 2021Q4
Data processing complete. Displaying DataFrame:


Year        PM25
Country City                            
AE      Dubai           2015  118.714286
AT      Linz            2015   53.005376
        Vienna          2015   52.629630
AU      Brisbane        2015   24.947090
        Darwin          2015   26.154286
...                      ...         ...
ZA      Port Elizabeth  2021   31.040816
        Pretoria        2021   51.428571
        Richards Bay    2021   17.846154
        Vereeniging     2021   42.428571
        Worcester       2021   23.021739

[15444 rows x 2 columns]

In [7]:
# Extract US-only data from the main DataFrame
us_airquality_df = reduced_df.loc['US']

# List all unique US cities present in the data
us_cities = us_airquality_df.index.tolist()
pprint(us_cities)

['Albuquerque',
 'Atlanta',
 'Austin',
 'Baltimore',
 'Boise',
 'Boston',
 'Brooklyn',
 'Charlotte',
 'Chicago',
 'Columbia',
 'Columbus',
 'Dallas',
 'Denver',
 'Detroit',
 'El Paso',
 'Fort Worth',
 'Fresno',
 'Hartford',
 'Honolulu',
 'Houston',
 'Indianapolis',
 'Jackson',
 'Jacksonville',
 'Las Vegas',
 'Little Rock',
 'Los Angeles',
 'Madison',
 'Manhattan',
 'Memphis',
 'Miami',
 'Milwaukee',
 'Nashville',
 'Oakland',
 'Oklahoma City',
 'Omaha',
 'Philadelphia',
 'Phoenix',
 'Portland',
 'Providence',
 'Queens',
 'Raleigh',
 'Richmond',
 'Sacramento',
 'Saint Paul',
 'Salem',
 'Salt Lake City',
 'San Antonio',
 'San Diego',
 'San Francisco',
 'San Jose',
 'Seattle',
 'Springfield',
 'Staten Island',
 'Tallahassee',
 'The Bronx',
 'Tucson',
 'Washington D.C.',
 'Albuquerque',
 'Atlanta',
 'Austin',
 'Baltimore',
 'Boise',
 'Boston',
 'Brooklyn',
 'Charlotte',
 'Chicago',
 'Columbia',
 'Columbus',
 'Dallas',
 'Denver',
 'Detroit',
 'El Paso',
 'Fort Worth',
 'Fresno',
 'Hartford',

In [9]:
from country_codes import get_state  # Import function to determine state from city

# Add a 'State' column by mapping each city to its corresponding state
for index, row in us_airquality_df.iterrows():
    us_airquality_df.loc[index, 'State'] = get_state(index)

# Select and reorder columns for clarity
us_airquality_df = us_airquality_df[['State', 'Year', 'PM25']]

# Display the resulting DataFrame for verification
us_airquality_df

/var/folders/1g/q9gxtv_j0vxc7tlf0j1712sh0000gn/T/ipykernel_67569/2421444263.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_airquality_df.loc[index, 'State'] = get_state(index)
/var/folders/1g/q9gxtv_j0vxc7tlf0j1712sh0000gn/T/ipykernel_67569/2421444263.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'New Mexico' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  us_airquality_df.loc[index, 'State'] = get_state(index)


,State,Year,PM25
City,,,
Albuquerque,New Mexico,2015,24.054545
Atlanta,Georgia,2015,41.853261
Austin,Minnesota,2015,32.465608
Baltimore,Maryland,2015,42.973262
Boise,Idaho,2015,32.529101
...,...,...,...
Staten Island,New York,2021,32.540816
Tallahassee,Florida,2021,34.418367
The Bronx,New York,2021,29.246575


In [10]:
# Group by state and year, calculating the mean PM2.5 for each combination
states_airquality_df = us_airquality_df.groupby(['State', 'Year']).mean('PM25')

# Save the grouped data to a CSV file
states_airquality_df.to_csv("Cleaned_Data/cleaned_airquality_usstates.csv", index=True)
